In [7]:
from selenium import webdriver
import pandas as pd
from sqlalchemy import create_engine
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


#Variables

In [8]:
page=1
url = 'https://zyntern.com/jobs?page='+str(page)
hirdetesLeiras = {
    'Indicator' : [],
    'ceg': [],
    'pozicio' :[],
    'skills': [],
    'soft skills' : [] ,
    'details' : [],
    'bottom-location': [],
    'link' : []
 }
databasename = str(pd.to_datetime("today").date()) +' zynter_com.db'
engine = create_engine('sqlite:///' + databasename, echo=False)


In [9]:

chrome_options = Options()
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--log-level=3")
driver = webdriver.Chrome(options=chrome_options) #options=chrome_options
driver.get(url)

# A szükséges információk összegyüjtése.

In [10]:
allasokSzama = driver.find_elements(By.TAG_NAME ,'h1')[0].text #állások száma

# Fügvény a declarált dict feltöltésére a kinyert adatokkal.

In [11]:
def dictappend(hirdetesekKontener):
    for i in range(len(hirdetesekKontener)):
        indexincrement = len(hirdetesLeiras['Indicator'])
        hirdetesLeiras['Indicator'].append(indexincrement)
        hirdetesLeiras['link'].append(hirdetesekKontener[i].get_attribute('href'))
        job = hirdetesekKontener[i].find_elements_by_xpath('./div')
        ceg = hirdetesekKontener[i].find_elements_by_xpath('./div/h5')[0]
        poz =hirdetesekKontener[i].find_elements_by_xpath('./div/h3')[0]
        skills = hirdetesekKontener[i].find_elements(By.XPATH , './div/div[1]/span')
        softSkills = hirdetesekKontener[i].find_elements(By.XPATH , './div/div[2]/span')
        for j in range(len(job)):
            if job[j].get_attribute('class') == 'info':
                    hirdetesLeiras[ 'ceg'].append(ceg.text)
                    hirdetesLeiras['pozicio'].append(poz.text)
                    fs=""
                    fss=""
                    for i in range(len(skills)):
                        fs += skills[i].text 
                    hirdetesLeiras['skills'].append(fs)
                    for i in range(len(softSkills)):
                        fss += softSkills[i].text
                    hirdetesLeiras['soft skills'].append(fss)
            elif   job[j].get_attribute('class') == 'details'  :
                    hirdetesLeiras[ 'details'].append(job[j].text.replace("\n" , " "))
            elif  job[j].get_attribute('class') =='bottom-location':
                hirdetesLeiras[ 'bottom-location'].append(job[j].text)

# Az oldal megnyitása , a poziciók adatainak kigyüjtése , majd oldalváltás

In [12]:
#Az oldalon addig lapozni amig az utolsó oldalt el nem éri .
nincstobboldal = driver.find_elements(By.XPATH ,"//*[contains(text() , 'Nincs találat')]" )


while len(nincstobboldal) == 0 : 
    url2 = 'https://zyntern.com/jobs?page='+str(page)
    driver.get(url2)
    driver.implicitly_wait(5)
    hirdetesekKontener = driver.find_elements(By.CSS_SELECTOR ,'#job_listing_container > div > div > div.listings>a')
    dictappend(hirdetesekKontener)
    page=page+1
    nincstobboldal = driver.find_elements(By.XPATH ,"//*[contains(text() , 'Nincs találat')]" )
    print(url2)
    print(f"Eddig kinyert adat : {len(hirdetesLeiras['Indicator'])} ")
print(f"Az oldalon talált állások száma : {allasokSzama} ebből a mennyiségből kinyert addat : {len(hirdetesLeiras['Indicator'])} ")  

https://zyntern.com/jobs?page=1
Eddig kinyert adat : 12 
https://zyntern.com/jobs?page=2
Eddig kinyert adat : 24 
https://zyntern.com/jobs?page=3
Eddig kinyert adat : 36 
https://zyntern.com/jobs?page=4
Eddig kinyert adat : 48 
https://zyntern.com/jobs?page=5
Eddig kinyert adat : 60 
https://zyntern.com/jobs?page=6
Eddig kinyert adat : 72 
https://zyntern.com/jobs?page=7
Eddig kinyert adat : 84 
https://zyntern.com/jobs?page=8
Eddig kinyert adat : 96 
https://zyntern.com/jobs?page=9
Eddig kinyert adat : 108 
https://zyntern.com/jobs?page=10
Eddig kinyert adat : 120 
https://zyntern.com/jobs?page=11
Eddig kinyert adat : 132 
https://zyntern.com/jobs?page=12
Eddig kinyert adat : 144 
https://zyntern.com/jobs?page=13
Eddig kinyert adat : 156 
https://zyntern.com/jobs?page=14
Eddig kinyert adat : 168 
https://zyntern.com/jobs?page=15
Eddig kinyert adat : 180 
https://zyntern.com/jobs?page=16
Eddig kinyert adat : 192 
https://zyntern.com/jobs?page=17
Eddig kinyert adat : 204 
https://zynter

# A kinyert addat mentése XLSX és SQLlite3 db formátumba.

In [13]:
df = pd.DataFrame(hirdetesLeiras)
df = df.drop('Indicator', axis=1)
df.to_excel(str(pd.to_datetime("today").date()) + ' zyntern_com.xlsx')
df.to_sql('hirdetesLeiras', con=engine, if_exists='append')
driver.quit()